In [ ]:
import glob
import math
import os
import re
import sklearn
import sys

import pandas as pd
import numpy as np

from itertools import compress
from sklearn.utils import shuffle 
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, \
accuracy_score, roc_curve, precision_recall_curve
from sklearn.ensemble import RandomForestClassifier


In [ ]:
class ASBException(Exception):
    pass

class ASBIndex:
    """
    Object to work with the information stored in the database
    
    Replacement for the hdf to be able to use 7z zipping
    """
    
    SPECIFIC_FEATURES = {'motif_log_pref', 'motif_log_palt', 'motif_fc', 'motif_pos', 'is_asb'}

    def __init__(self, base_ids, base_dir, specific_features=None):
        """
        Initialize object. 
        Should not be run directly. Objects of ASBIndex must be create through class methods
        
        Parameters
        ----------
        base_ids: list
            SNV ids in the database
        base_dir: str
            base directory with SNV features. Each feature is stored in the vector with each element corresponding to snv in base_ids
        specific_features: set[str] - set of dataset-specific features
        """
        self.base_ids = base_ids
        self.base_dir = base_dir
        
        specific_features = specific_features or ASBIndex.SPECIFIC_FEATURES
        self.specific_features = specific_features

    def retrieve_datatable(self, 
                           query_ids, 
                           query_feat, 
                           dataset_name=None):
        """ retrieve infromation about SNV ids
        Parameters
        ----------
        query_ids: list[str] - SNV ids to get information about
        query_feat: list[str] - features to retrieve
        dataset_name: str - name of the dataset to retrieve. Required if any dataset-specific features are queried
        Returns
        -------
        pd.DataFrame
            Table containing information about quered ids
        """
        
        q_set = set(query_ids)
        mask = np.array([(q in q_set) for q in self.base_ids])
        
        if mask.sum() != len(query_ids):
            not_found = q_set - set(self.base_ids)
            if not_found:
                raise ASBException(f"Ids weren't found: {', '.join(q_set - set(self.base_ids))}")
            else:
                k, c = np.unique(query_ids, return_counts=True)
                k = k[c > 1]
                if k.shape[0] > 0:
                    raise ASBException(f"Duplicated ids: {', '.join(list(k))}")
                else:
                    raise ASBException("Uknown error occured")
                
                
        order = {k : i for i, k in enumerate(query_ids)}
        dt = {}
    
        for col in query_feat:
            try:
                col_path = self.get_feat_path(col, dataset_name)
                col_ar = np.load(col_path, allow_pickle=True)
            except FileNotFoundError:
                if col in self.specific_features:
                    print(f"File corresponding to dataset-specific feature '{col}' from dataset '{dataset_name}' doesn't exist", 
                      file=sys.stderr)
                else:
                    print(f"File corresponding to feature '{col}' doesn't exist", file=sys.stderr)
            else:
                dt[col] = col_ar[mask]

        dt =  pd.DataFrame(dt, columns=dt.keys())
        dt['ID'] = list(compress(self.base_ids, mask))
        dt['ID_mapped'] = dt['ID'].apply(lambda x: order[x])   # to run on Python3.6
        dt.sort_values(by="ID_mapped", axis=0, inplace=True)
        dt.drop("ID_mapped", axis=1, inplace=True)
        dt.set_index("ID", inplace=True)
        
        return dt
    
    
    def add_columns_from_tab(self, 
                             tab,
                             dataset_name=None,
                             ID_name="index"):
        """
        Add features from the datatable to the database
        
        Parameters
        ----------
        tab: pd.DataFrame - table containing features to add
        dataset_name: str -  name of the dataset. Required if any dataset-specific features are added
        ID_name: str - name of column containing ids, if ID_name == "index" table index will be used
        """
        if ID_name != "index":
            tab = tab.set_index(ID_name)
        
        
        for col in tab.columns:
            path = self.get_feat_path(col, dataset_name)
            if os.path.exists(path):
                if col in self.specific_features:
                    raise ASBException(f"Dataset specific feature {col} for '{dataset_name}' dataset already exists"
                                       f", specify other `dataset_name` to add it to the table")
                else:
                    raise ASBException(f"Feature {col} already exists")
        
        ext_tab = tab.reindex(self.base_ids)
        
        for col in ext_tab.columns:
            out = self.get_feat_path(col, dataset_name)
            np.save(out, ext_tab[col])
            
    def get_index_path(self):
        """
        return path to the index file of the database
        """
        return os.path.join(self.base_dir, "id.lst")
    
    def get_feat_path(self, feat, dataset_name=None):
        """
        return path to the feature in the database
        
        Parameters
        ----------
        feat: str - name of the feature
        dataset_name: str - name of the dataset. Required if feature is dataset-specific
        """
        
        if feat in self.specific_features:
            if dataset_name is None:
                raise ASBException(f"Dataset name is required to get path for dataset-specific feature {feat}")
            feat_name = f"{feat}_{dataset_name}"
        else:
            feat_name = feat
        return os.path.join(self.base_dir, f"{feat_name}.npy")
    
    def get_specific_features_path(self):
        """
        return path to the file with list of dataset-specific features
        """
        return os.path.join(self.base_dir, "specific_features.lst")
    
    
    def get_subset_ids_dir(self):
        """
        return path to directory storing ids subsets
        """
        
        return os.path.join(self.base_dir, "ids")
    
    def get_subset_feats_dir(self):
        """
        return path to directory storing features subsets
        """
        
        return os.path.join(self.base_dir, "feats")
    
    def get_ids_subset_path(self, name):
        """
        return path to the file with ids subset
        
        Paramters
        ---------
        name - subset name
        """
        
        return os.path.join(self.get_subset_ids_dir(), f"{name}.lst")
    
    
    def get_feats_subset_path(self, name):
        """
        return path to the file with features subset 
        
        Paramters
        ---------
        name - subset name
        """
        
        return os.path.join(self.get_subset_feats_dir(), f"{name}.lst")
    
    
    def write_index(self):
        """
        write database index file
        """
        with open(self.get_index_path(), "w") as out:
            out.write("\n".join(self.base_ids))
            
    def write_specific_features(self):
        """
        write dataset-specific features file
        """
        with open(self.get_specific_features_path(), "w") as out:
            out.write("\n".join(self.specific_features))
    
    
    def make_basedir(self):
        """
        make database basedir
        """
        if os.path.exists(self.base_dir):
            raise ASBException(f"{self.base_dir} already exists")
        os.makedirs(self.base_dir)
        ids_path = os.path.join(self.base_dir, "ids")
        os.makedirs(self.get_subset_ids_dir())
        os.makedirs(self.get_subset_feats_dir())
            
            
    @classmethod
    def create(cls, base_ids, base_dir, specific_features=None):
        """
        Create empty database to store features of `base_ids`
        
        Parameters
        ----------
        base_ids: list
            SNV ids in the database
        base_dir: str
            base directory with SNV features. 
            Each feature is stored in the vector with each element corresponding to snv in base_ids
        specific_features: set[str] 
            set of dataset-specific features
        """
        specific_features = specific_features or ASBIndex.SPECIFIC_FEATURES
        self = cls(base_ids, base_dir, specific_features.copy())
        self.make_basedir()
        self.write_index()
        self.write_specific_features()
        return self
        
                
    @classmethod
    def from_table(cls, 
                   tab,
                   base_dir, 
                   dataset_name=None,
                   specific_features=None,
                   ID_name="index"):
        """
        Create database from a datatable
        
        Parameters
        ----------
        tab: pd.DataFrame - table to create database from 
        basedir: str - path to database base directory
        dataset_name: str - name of the dataset, required if any feature in the table is dataset-specific
        ID_name: str - name of ids column
        """ 
        
        specific_features = specific_features or ASBIndex.SPECIFIC_FEATURES
        if ID_name != "index":
            base_ids = list(map(str, tab[ID_name]))
        else:
            base_ids = list(map(str, tab.index))
    
        self = ASBIndex.create(base_ids, base_dir, specific_features.copy())

        for col in tab.columns:
            out = self.get_feat_path(col, dataset_name)
            np.save(out, tab[col])
            
        return self
    
    @classmethod
    def load(cls, base_dir):
        """
        Load  database from dir
        
        Parameters
        ----------
        basedir: str - path to database base directory
        """
        self = cls([], base_dir)
        
        with open(self.get_index_path(), "r") as inp:
            base_ids = [line.strip() for line in inp]
            
        with open(self.get_specific_features_path(), "r") as inp:
            specific_features = set(line.strip() for line in inp)
        
        self.base_ids = base_ids
        self.specific_features = specific_features
        return self
    
    def define_ids_subset(self, ids, name):
        ids_path = self.get_ids_subset_path(name)
        if os.path.exists(ids_path):
            raise ASBException(f"ids subset with name '{name}' already exists")
        with open(ids_path, "w") as out:
            out.write("\n".join(ids))
            
    def load_ids_subset(self, name):
        ids_path = self.get_ids_subset_path(name)
        if not os.path.exists(ids_path):
            raise ASBException(f"ids subset with name '{name}' doesn't exist")
        with open(ids_path, "r") as inp:
            ids = [line.strip() for line in inp]
        return ids 
    
    def define_feat_subset(self, feats, name):
        feats_path = self.get_feats_subset_path(name)
        if os.path.exists(feats_path):
            raise ASBException(f"features subset with name '{name}' already exists")
        with open(feats_path, "w") as out:
            out.write("\n".join(feats))

    def load_feat_subset(self, name):
        feats_path = self.get_feats_subset_path(name)
        if not os.path.exists(feats_path):
            raise ASBException(f"features subset with name '{name}' doesn't exist")
        with open(feats_path, "r") as inp:
            feats = [line.strip() for line in inp]
        return feats
        
                                    
    def add_ids(self):
        raise NotImplementedError()
    
    def add_specific_feature(self):
        raise NotImplementedError()

In [ ]:
def df_shuffle(df, 
               copy=True,
               random_state=1):
    '''
    shuffle values in dataframe columnwise
     
    Parameters 
    ----------
    df: pd.DataFrame - dataframe
    copy: bool - copy dataframe or do shuffle inplace
    random_state: int 
    '''
    if copy:
        df = df.copy()
    for column in df.columns:
        df.loc[:, column] = shuffle(df[[column]].values, random_state=random_state)
    return df

In [ ]:
def random_feature_importance(clf,
                              df, 
                              label_col="is_asb", 
                              shuffle_features=True, 
                              random_state=1):
    '''
    get random feature importance (for shuffled input features)
    Parameters
    ----------
    clf: classifier - classifier with sklearn-friendly interface  
    df: pd.DataFrame - dataframe for training  
    label_col: - labels column name
    shuffle_features: bool - shuffle features or shuffle labels
    random_state: int
    '''

    X = df.drop(columns=[label_col])
    y = df[label_col]

    if shuffle_features:
        X = df_shuffle(X, copy=False, random_state=random_state)
    else:
        y = shuffle(df[label_col], random_state=random_state)
    
    clf.fit(X, y)
    max_rnd_importance = np.max(clf.feature_importances_)
    return max_rnd_importance

In [ ]:
def write_table(df, filepath):
    '''
    Write pandas dataframe in tab-separated format

    Parameters
    ----------
    df: pd.DataFrame - dataframe
    filepath: str - path to write
    '''
    df.to_csv(filepath, sep="\t", index=False)

In [ ]:
def curves(y_true, pred_prob):
    '''
    calculate points for rocauc and prauc curves
    Parameters 
    ----------
    y_true: np.array - vector of true labels 
    pred_prob: np.array - vector of probabilities for class 1
    '''
    fpr, tpr, thresholds = roc_curve(y_true, pred_prob)
    pr, rec, thresh_prauc = precision_recall_curve(y_true, pred_prob)
    df_roc = pd.DataFrame(data={'fpr': fpr, 
                                'tpr': tpr, 
                                'thresholds_roc': thresholds})
    df_pr = pd.DataFrame(data={'precision': pr[:-1], 
                               'recall': rec[:-1], 
                               'thresholds_prauc': thresh_prauc})
    return df_roc, df_pr

In [ ]:
def cross_validation_chromosomes(clf,
              data, 
              importance_threshold,
              ids_subsets=None, 
              chr_label="chr",
              y_label="is_asb"):
    '''
    clf: classifer
    data: pd.DataFrame - dataset for chromosomewise crossvalidation
    importance_threshold: float - threshold to determine important features
    ids_subsets: Union[dict, None] - subsets to calculate validations scores on.
    chr_label: str - name of chromosome column in the dataset
    y_label: str - name of label column in the dataset
    '''
    df_roc_lst = []
    df_pr_lst = []
    dt = {"val_chr": [], 
          "roc_auc": [], 
          "f1": [], 
          "accuracy": [],
          "pr_auc": [], 
          "important_features": [],
          "feature_scores": [],
          "subset_name": []}

    if ids_subsets is None:
        ids_subsets = {}

    ids_subsets['all'] = None

    for i in range(1, 23):
        val_chr = f"chr{i}"
        
        df_train = data[data[chr_label] != val_chr]
        df_val = data[data[chr_label] == val_chr]
        if (df_train.shape[0] <= 1 or df_val.shape[0] <= 1): # no data for chromosome i or only one class
            print(f"Skipping validation for chromosome {val_chr} - not enought data")
            continue
        
        X_train = df_train.drop(columns = [y_label, chr_label])
        X_val = df_val.drop(columns = [y_label, chr_label])
        y_train = df_train[y_label]
        y_val = df_val[y_label]
        if len(np.unique(y_train)) == 1 or len(np.unique(y_val)) == 1: # only one class for chromosome i
            print(f"Skipping validation for chromosome {val_chr} - not enought data")
            continue

        clf.fit(X_train, y_train)

        for subset_name, ids in ids_subsets.items():
            dt['val_chr'].append(val_chr)
            dt['subset_name'].append(subset_name)

            if subset_name != "all":
                X_val_part = X_val.reindex(ids).dropna()
                y_val_part = y_val.reindex(ids).dropna()
            else:
                X_val_part = X_val
                y_val_part = y_val
          
            if len(np.unique(y_val_part)) < 2:
                print(f"Skipping validation for chromosome {val_chr} on subset {subset_name}- not enought data",
                      file=sys.stderr)
                dt['roc_auc'].append(None)
                dt['f1'].append(None)
                dt['accuracy'].append(None)
                dt['pr_auc'].append(None)
                dt['important_features'].append(None)
                dt['feature_scores'].append(None)
                continue

            

            rfpred = clf.predict_proba(X_val_part)[:, 1]
            pred_classes = clf.predict(X_val_part)
            dt['roc_auc'].append(roc_auc_score(y_val_part, rfpred))
            dt['f1'].append(f1_score(y_val_part, pred_classes))
            dt['accuracy'].append(accuracy_score(y_val_part, pred_classes))
            dt['pr_auc'].append(average_precision_score(y_val_part, rfpred)) 
            
            imp_mask = clf.feature_importances_ > importance_threshold
            dt['important_features'].append(list(X_train.columns[imp_mask]))
            dt['feature_scores'].append(clf.feature_importances_[imp_mask])
            
            df_roc_part, df_pr_part = curves(y_val_part, rfpred)
            df_roc_part['val_chr'] = val_chr
            df_roc_part['subset_name'] = subset_name
            df_pr_part['val_chr'] = val_chr
            df_pr_part['subset_name'] = subset_name
            df_roc_lst.append(df_roc_part)
            df_pr_lst.append(df_pr_part)


    df_qual = pd.DataFrame(dt, columns=dt.keys())
    df_roc = pd.concat(df_roc_lst)
    df_pr = pd.concat(df_pr_lst)

    return df_qual, df_roc, df_pr

# Downloading data 

In [ ]:
%%bash
pip3 install wldhx.yadisk-direct
wget -O asb_data.7z $(yadisk-direct https://yadi.sk/d/u61gSHWT1wHkxw)
7za x asb_data.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
1 file, 2849142134 bytes (2718 MiB)

Extracting archive: asb_data.7z
--
Path = asb_data.7z
Type = 7z
Physical Size = 2849142134
Headers Size = 28817
Method = LZMA2:24
Solid = +
Blocks = 7

Everything is Ok

Folders: 3
Files: 4697
Size:       8654630213
Compressed: 2849142134


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Global vars

In [ ]:
CELL_LINE_NAMES = ["foreskin_keratinocyte", 
                   "HepG2__hepatoblastoma_",
                   "LNCaP__prostate_carcinoma_",
                   "K562__myelogenous_leukemia_",
                   "A549__lung_carcinoma_",
                   "GM12878__female_B-cells_",
                   "VCaP__prostate_carcinoma_",
                   "CD14+_monocytes",
                   "HEK293__embryonic_kidney_",
                   "MCF7__Invasive_ductal_breast_carcinoma_"]
TF_NAMES = ["SPI1",
            "CTCF",
            "FOXK2",
            "STAT1",
            "FOXA1",
            "IKZF1",
            "ESR1",
            "RAD21",
            "ANDR",
            "CEBPB"]

N_ESTIMATORS = 500
RANDOM_STATE = 1
N_JOBS = -1

# Example: Training individual TF-classifier 

In [ ]:
tf_name = TF_NAMES[0]
print(tf_name)
index = ASBIndex.load("asb_data")
feat = index.load_feat_subset("TF") + ['is_asb']
feat = list(set(feat) - {'pos', 'ref', 'alt'})
ids = index.load_ids_subset(tf_name)
df = index.retrieve_datatable(ids, feat, tf_name)

SPI1


In [ ]:
max_rnd_importance = random_feature_importance(RandomForestClassifier(n_estimators=N_ESTIMATORS,
                                                                      n_jobs=N_JOBS,
                                                                      random_state=RANDOM_STATE),
                              df.drop(columns=["chr"]),  
                              random_state=RANDOM_STATE)

df_qual, df_roc, df_pr = cross_validation_chromosomes(RandomForestClassifier(n_estimators=N_ESTIMATORS,
                                                                      n_jobs=N_JOBS,
                                                                      random_state=RANDOM_STATE), 
                                                      df,
                                                      max_rnd_importance)

# Training individual Cellline-classifier

In [ ]:
cline_name = CELL_LINE_NAMES[0] 
print(cline_name)
index = ASBIndex.load("asb_data")
feat = index.load_feat_subset("CL") + ['is_asb']
feat = list(set(feat) - {'pos', 'ref', 'alt'})
ids = index.load_ids_subset(cline_name)
df = index.retrieve_datatable(ids, feat, cline_name)

In [ ]:
max_rnd_importance = random_feature_importance(RandomForestClassifier(n_estimators=N_ESTIMATORS,
                                                                      n_jobs=N_JOBS,
                                                                      random_state=RANDOM_STATE),
                              df.drop(columns=["chr"],  
                              random_state=RANDOM_STATE)

df_qual, df_roc, df_pr = cross_validation_chromosomes(RandomForestClassifier(n_estimators=N_ESTIMATORS,
                                                                      n_jobs=N_JOBS,
                                                                      random_state=RANDOM_STATE),
                                                      df,
                                                      max_rnd_importance)

# Training global TF classifier

Warning: This and next example may require running on server, not Colab

In [ ]:
tf_ids = {name: index.get_ids_subset_path(name) for name in TF_NAMES}

In [ ]:
feat = index.load_feat_subset("TF") + ['is_asb']
feat = list(set(feat) - {'pos', 'ref', 'alt'})
ids = index.base_ids
df = index.retrieve_datatable(ids, feat, "TF_joined")

In [ ]:
max_rnd_importance = random_feature_importance(RandomForestClassifier(n_estimators=N_ESTIMATORS,
                                                                      n_jobs=N_JOBS,
                                                                      random_state=RANDOM_STATE),
                              df.drop(columns=["chr"],  
                              random_state=RANDOM_STATE)
df_qual, df_roc, df_pr = cross_validation_chromosomes(RandomForestClassifier(n_estimators=N_ESTIMATORS,
                                                                      n_jobs=N_JOBS,
                                                                      random_state=RANDOM_STATE), 
                                                      df,
                                                      max_rnd_importance,
                                                      ids_subsets=tf_ids)

## Training global Celline classifier

In [ ]:
cl_ids = {name: index.get_ids_subset_path(name) for name in CELL_LINE_NAMES}

In [ ]:
feat = index.load_feat_subset("CL") + ['is_asb']
feat = list(set(feat) - {'pos', 'ref', 'alt'})
ids = index.base_ids
df = index.retrieve_datatable(ids, feat, "CL_joined")

In [ ]:
max_rnd_importance = random_feature_importance(RandomForestClassifier(n_estimators=N_ESTIMATORS,
                                                                      n_jobs=N_JOBS,
                                                                      random_state=RANDOM_STATE),
                                               df.drop(columns=["chr"],  
                                               random_state=1)

df_qual, df_roc, df_pr = cross_validation_chromosomes(RandomForestClassifier(n_estimators=N_ESTIMATORS,
                                                                      n_jobs=N_JOBS,
                                                                      random_state=RANDOM_STATE), 
                                                      df,
                                                      max_rnd_importance,
                                                      ids_subsets=cl_ids)